
# Project 1: Build an LLM Playground

Welcome! In this project, you’ll learn foundations of large language models (LLMs). We’ll keep the code minimal and the explanations high‑level so that anyone who can run a Python cell can follow along.  

We'll be using Google Colab for this project. Colab is a free, browser-based platform that lets you run Python code and machine learning models without installing anything on your local computer. Click the button below to open this notebook directly in Google Colab and get started!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects/blob/main/project_1/lm_playground.ipynb)

---
## Learning Objectives  
* **Tokenization** and how raw text is tokenized into a sequene of discrete tokens
* Inspect **GPT2** and **Transformer architecture**
* Loading pre-trained LLMs using **Hugging Face**
* **Decoding strategies** to generate text from LLMs
* Completion versus **intrusction fine-tuned** LLMs


Let's get started!

In [ ]:
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)

# 1 - Tokenization

A neural network can’t digest raw text. They need **numbers**. Tokenization is the process of converting text into IDs. In this section, you'll learn how tokenization is implemented in practice.

Tokenization methods generally fall into three categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word‑level tokenization

Split text on whitespace and store each **word** as a token.

In [1]:
# 1. Tiny corpus
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# 2. Build the vocabulary
PAD, UNK = "[PAD]", "[UNK]"
vocab = [PAD, UNK]
word2id = {PAD: 0, UNK: 1}
id2word = {0: PAD, 1: UNK}

"""
YOUR CODE HERE
"""
current_id = 2
for line in corpus:
  for word in line.split(" "):
    if word not in vocab:
      vocab.append(word)
      word2id[word] = current_id
      id2word[current_id] = word
      current_id += 1

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])

# 3. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    result = []
    for word in text.split(" "):
      if word in word2id:
        id = word2id[word]
      else:
        id = word2id[UNK]
      result.append(id)

    return result

def decode(ids):
    """
    YOUR CODE HERE
    """
    result = []
    for id in ids:
      if id in id2word:
        word = id2word[id]
      else:
        word = UNK
      result.append(word)

    return " ".join(result)

# 4. Demo
sample = "The brown unicorn jumps"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)

print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 22 words
First 15 vocab entries: ['[PAD]', '[UNK]', 'The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', 'Tokenization', 'converts', 'text', 'to']

Input text : The brown unicorn jumps
Token IDs  : [2, 4, 1, 6]
Decoded    : The brown [UNK] jumps


Word-level tokenization has two major limitations:
1. Large vocabulary size
2. Out-of-vocabulary (OOV) issue

### 1.2 - Character‑level tokenization

Every single character (including spaces and emojis) gets its own ID. This guarantees zero out‑of‑vocabulary issues but very long sequences.

In [2]:
# 1. Build a fixed vocabulary # a–z + A–Z + padding + unkwown
import string

vocab = []
char2id = {}
id2char = {}
"""
YOUR CODE HERE
"""
vocab.append("[PAD]")
vocab.append("[UNK]")
for char in string.ascii_lowercase:
  vocab.append(char)
  char2id[char] = len(vocab) - 1
  id2char[len(vocab) - 1] = char
  vocab.append(char.upper())
  char2id[char.upper()] = len(vocab) - 1
  id2char[len(vocab) - 1] = char.upper()
print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    result = []
    for char in text:
      if char in char2id:
        id = char2id[char]
      else:
        id = char2id[UNK]
      result.append(id)

    return result

def decode(ids):
    """
    YOUR CODE HERE
    """
    result = []
    for id in ids:
      if id in id2char:
        char = id2char[id]
      else:
        char = UNK
      result.append(char)

    return "".join(result)

# 3. Demo
sample = "Hello"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Decoded    :", recovered)

Vocabulary size: 54 (52 letters + 2 specials)

Input text : Hello
Token IDs  : [17, 10, 24, 24, 30]
Decoded    : Hello


### 1.3 - Subword‑level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** the most common character and gorup them into new tokens. For example, the word `unbelievable` might turn into three tokens: `["un", "believ", "able"]`. This approach strikes a balance between word-level and character-level methods and fix their limitations.

For example, `BPE` algorithm forms the vocabulary using the following steps:
1. **Start with bytes** → every character is its own token.  
2. **Count all adjacent pairs** in a huge corpus.  
3. **Merge the most frequent pair** into a new token.  
   *Repeat steps 2-3* until you hit the target vocab size (e.g., 50 k).

Let's see `BPE` in practice.

In [3]:
# 1. Load a pretrained BPE tokenizer (GPT-2 uses BPE).
# Refer to  https://huggingface.co/docs/transformers/en/fast_tokenizers

from transformers import AutoTokenizer

bpe_tok = AutoTokenizer.from_pretrained("gpt2")

# 2. Encode / decode
def encode(text):
    """
    YOUR CODE HERE
    """
    bpe_tokens = bpe_tok.tokenize(text)
    ids = bpe_tok.convert_tokens_to_ids(bpe_tokens)
    return ids


def decode(ids):
    """
    YOUR CODE HERE
    """
    bpe_tokens = bpe_tok.convert_ids_to_tokens(ids)
    text = bpe_tok.convert_tokens_to_string(bpe_tokens)
    return text

# 3. Demo
sample = "Unbelievable tokenization powers! 🚀"
ids = encode(sample)
recovered = decode(ids)

print("\nInput text :", sample)
print("Token IDs  :", ids)
print("Tokens     :", bpe_tok.convert_ids_to_tokens(ids))
print("Decoded    :", recovered)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


Input text : Unbelievable tokenization powers! 🚀
Token IDs  : [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens     : ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']
Decoded    : Unbelievable tokenization powers! 🚀


### 1.4 - TikToken

`tiktoken` is a production-ready library which offers high‑speed tokenization used by OpenAI models.  
Let's compare the older **gpt2** encoding with the newer **cl100k_base** used in GPT‑4.

In [5]:
# Use gpt2 and cl100k_base to encode and decode the following text
# Refer to https://github.com/openai/tiktoken
import tiktoken

sentence = "The 🌟 star-player scored 40 points!"

"""
YOUR CODE HERE
"""
gpt2_tok = tiktoken.get_encoding("gpt2")
cl100k_base_tok = tiktoken.get_encoding("cl100k_base")

Experiment: try new sentences, emojis, code snippets, or other languages. If you are interested, try implementing the BPE algorithm yourself.

### 1.5 - Key Takeaways

* **Word‑level**: simple but brittle (OOV problems).  
* **Character‑level**: robust but produces long sequences.  
* **BPE / Byte‑Level BPE**: middle ground used by most LLMs.  
* **tiktoken**: shows how production models tokenize with pre‑trained sub‑word vocabularies.

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer applies a simple operation (matrix multiplication, attention, etc.). Stacking hundreds of these layers lets the model capture patterns and statistical relations from text. The final output is a vector of scores that says, “how likely is each possible token to come next?”

> Think of the whole network as **one gigantic equation** whose parameters were tuned during training to minimize prediction error.



### 2.1 - A Single `Linear` Layer

Before we explore Transformer, let’s start tiny:

* A **Linear layer** performs `y = Wx + b`  
  * `x` – input vector  
  * `W` – weight matrix (learned)  
  * `b` – bias vector (learned)

Although this looks basic, chaining thousands of such linear transforms (with nonlinearities in between) gives neural nets their expressive power.


In [6]:
import torch
import torch.nn as nn

class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        return x @ self.weight.t() + self.bias

In [7]:
import torch.nn as nn, torch

lin = nn.Linear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[ 0.0472, -0.2928, -0.2223],
        [ 0.0434, -0.2523,  0.4697]], requires_grad=True)
Bias   : Parameter containing:
tensor([-0.1375,  0.5626], requires_grad=True)
Output : tensor([0.0914, 1.0931], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

Below, we load the smallest public GPT-2 (124 M parameters), grab its *first* block, and inspect the pieces.


In [8]:
import torch
from transformers import GPT2LMHeadModel

# Load the 124 M-parameter GPT-2 and inspect its layers (12 layers)
"""
YOUR CODE HERE
"""
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
print(f"transformer blocks: {len(gpt2.transformer.h)}")
print(gpt2)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

transformer blocks: 12
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [9]:
# ========================================
# CHECK GPT-2 MODEL AND VERIFY 124M PARAMETERS
# ========================================

# 1. Check which pre-trained model name was used
print("=" * 70)
print("GPT-2 MODEL VERIFICATION")
print("=" * 70)
print("\nModel loaded using: 'gpt2'")
print("This should be the 124M parameter base GPT-2 model\n")

# 2. Inspect the model configuration
print("-" * 70)
print("MODEL CONFIGURATION:")
print("-" * 70)
print(f"  Hidden size (embedding dim): {gpt2.config.hidden_size}")
print(f"  Number of transformer layers: {gpt2.config.n_layer}")
print(f"  Number of attention heads: {gpt2.config.n_head}")
print(f"  Vocabulary size: {gpt2.config.vocab_size}")
print(f"  Max position embeddings: {gpt2.config.n_positions}\n")

# 3. Count total parameters in the model
total_params = sum(p.numel() for p in gpt2.parameters())
print("-" * 70)
print("PARAMETER COUNT:")
print("-" * 70)
print(f"  Total parameters: {total_params:,}")
print(f"  In millions: {total_params / 1e6:.2f}M\n")

# 4. Verification and conclusion
print("=" * 70)
if 120_000_000 <= total_params <= 130_000_000:
    print("✓ VERIFIED: This IS the 124M parameter GPT-2 model")
    print("✓ This MATCHES the assignment requirement!")
    print("\nConclusion: You are using the correct 124M-parameter")
    print("version of GPT-2 as required by the assignment.")
else:
    print("✗ WARNING: Parameter count does NOT match expected 124M")
    print(f"  Expected: ~124M, Got: {total_params / 1e6:.2f}M")
print("=" * 70)

GPT-2 MODEL VERIFICATION

Model loaded using: 'gpt2'
This should be the 124M parameter base GPT-2 model

----------------------------------------------------------------------
MODEL CONFIGURATION:
----------------------------------------------------------------------
  Hidden size (embedding dim): 768
  Number of transformer layers: 12
  Number of attention heads: 12
  Vocabulary size: 50257
  Max position embeddings: 1024

----------------------------------------------------------------------
PARAMETER COUNT:
----------------------------------------------------------------------
  Total parameters: 124,439,808
  In millions: 124.44M

✓ VERIFIED: This IS the 124M parameter GPT-2 model
✓ This MATCHES the assignment requirement!

Conclusion: You are using the correct 124M-parameter
version of GPT-2 as required by the assignment.


In [10]:
# Run a tiny forward pass through the first block
seq_len = 8
dummy_tokens = torch.randint(0, gpt2.config.vocab_size, (1, seq_len))
print("dummy_tokens: ", dummy_tokens) # a 1x8 tensor with 8 random integers in vocab size range

with torch.no_grad():
    # Embed tokens + positions the same way GPT-2 does
    # Forward through one layer
    """
    YOUR CODE HERE
    """
    # tokens
    token_embeddings = gpt2.transformer.wte(dummy_tokens)
    print("embeddings shape: ", token_embeddings.shape)
    print("embeddings: ", token_embeddings)

    # positions
    positions = torch.arange(seq_len, device=dummy_tokens.device).unsqueeze(0)
    print("positions: ", positions)
    position_embeddings = gpt2.transformer.wpe(positions)
    print("position embeddings shape: ", position_embeddings.shape)
    print("position embeddings: ", position_embeddings)

    # combine
    input_embeddings = token_embeddings + position_embeddings
    print("input embeddings shape: ", input_embeddings.shape)
    print("input embeddings: ", input_embeddings)

    # pass through transformer
    out = gpt2.transformer.h[0](input_embeddings)[0]
    print("output shape: ", out.shape)
    print("output: ", out)

print("\nOutput shape :", out.shape) # (batch, seq_len, hidden_size)

dummy_tokens:  tensor([[38769,  6455, 44573,  1599, 44286, 37188, 15246, 16031]])
embeddings shape:  torch.Size([1, 8, 768])
embeddings:  tensor([[[ 1.8955e-01,  2.7223e-02,  3.8846e-02,  ...,  3.0014e-02,
           9.5446e-02,  3.2155e-02],
         [-1.5363e-01, -1.5377e-01,  8.1620e-02,  ...,  4.4387e-02,
          -1.5313e-02, -3.6072e-01],
         [ 1.6074e-06, -1.4734e-01,  1.8892e-02,  ...,  6.8409e-04,
           8.8521e-02, -4.6554e-02],
         ...,
         [-5.6010e-02, -3.2159e-01, -6.6987e-03,  ...,  2.3542e-01,
           9.5411e-02, -6.5096e-02],
         [ 1.5315e-01,  1.8401e-02, -9.7815e-02,  ...,  1.1586e-01,
          -7.1665e-02,  2.0368e-01],
         [-2.0318e-02, -3.1693e-02,  2.5747e-01,  ..., -8.6417e-02,
           4.4556e-02, -6.2300e-02]]])
positions:  tensor([[0, 1, 2, 3, 4, 5, 6, 7]])
position embeddings shape:  torch.Size([1, 8, 768])
position embeddings:  tensor([[[-1.8821e-02, -1.9742e-01,  4.0267e-03,  ..., -4.3044e-02,
           2.8267e-02,  5.4

### 2.3 - Inside GPT-2

GPT-2 is just many of those modules arranged in a repeating *block*. Let's print the modules inside the Transformer.

In [11]:
# Print the name and modules inside gpt2

# print("do: ", gpt2.transformer.drop)
for name, module in gpt2.named_children():
  print(f"Module name: ", name)
  print(module)
  print("-" * 20)

Module name:  transformer
GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)
--------------------
Module name:  lm_head
Linear(in_features=768, out_features=50257, bias=False)
--------------------


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

Passing a token sequence through an **LLM** yields a tensor of **logits** with shape  
`(batch_size, seq_len, vocab_size)`.  
Applying `softmax` on the last dimension turns those logits into probabilities.

The cell below feeds an 8-token dummy sequence, prints the logits shape, and shows the five most likely next tokens for the final position.


In [12]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load gpt2 model and tokenizer
"""
YOUR CODE HERE
"""
# load the model
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

# load the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# disable dropouts since we're just doing inference
gpt2.eval()

# Tokenize input text
text = "I am the very model of a modern major"
"""
YOUR CODE HERE
"""
# tokenize the input, get a pytorch tensor as output to be used in next step (instead of a simple list of tokens)
tokens = tokenizer.encode(text, return_tensors="pt")
print("tokens: ", tokens)

# Get logits by passing the ids to the gpt2 model.
"""
YOUR CODE HERE
"""
# disable gradient tracking since we don't need backprop for inference
with torch.no_grad():
  # execute the model using the tokens
  output = gpt2(tokens)
  print("output shape: ", output)
  print("output: ", output)

  # get the logistic units from output (logits) to use as input to softmax
  logits = output.logits
  print("Logits shape :", logits.shape)

# Predict next token
"""
YOUR CODE HERE
"""
# each element is the logits representing what gpt2 would predict as the NEXT token if that index was the last of the input given
# so taking the logits of the LAST token of output gives us the data needed to calculate (using softmax) the next token prediction
# of the entire input sequence
last_token_logits = logits[0, -1, :]
print("Last token logits shape :", last_token_logits.shape)
print("Last token logits :", last_token_logits)

# take the last token logits and give it to softmax, which will give us the probabilities of each of the possible elements of the
# vocab (the logits won't necessarily add up to 1, but the softmax result will)
probs = F.softmax(last_token_logits, dim=-1)
print("Probabilities shape :", probs.shape)
print("Probabilities :", probs)

# just pick out the top 5 most likely next token indices (as well as their actual probabilities)
top_5_tokens = torch.topk(probs, 5)
print("Top-5 tokens :", top_5_tokens)

print("\nTop-5 predictions for the next token:")
"""
YOUR CODE HERE
"""
# iterate over top 5 most likely next tokens and output them with actual probability scores
for i in range(len(top_5_tokens.indices)):
  print("prediction: ", tokenizer.decode(top_5_tokens.indices[i]), ", probability: ", top_5_tokens.values[i])


tokens:  tensor([[  40,  716,  262,  845, 2746,  286,  257, 3660, 1688]])
output shape:  CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -39.3084,  -39.0100,  -41.8374,  ...,  -46.9337,  -44.9074,
           -39.5149],
         [-114.8939, -114.2495, -119.9518,  ..., -122.6206, -121.7605,
          -117.6996],
         [ -95.8772,  -93.9861,  -96.1859,  ...,  -98.2554,  -99.7003,
           -94.3344],
         ...,
         [-105.7121, -103.5980, -108.6309,  ..., -110.3217, -110.9370,
          -104.8304],
         [ -93.1989,  -90.3098,  -98.2595,  ..., -100.0687, -101.5967,
           -95.8216],
         [ -75.4937,  -74.8921,  -82.1228,  ...,  -85.4231,  -85.5668,
           -77.4899]]]), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None, cross_attentions=None)
output:  CausalLM

### 2.5 - Key Takeaway

A language model is nothing mystical: it’s a *huge composition* of small, understandable layers trained to predict the next token in a sequence of tokens.

# 3 - Generation
Once an LLM is trained to predict the probabilities, we can generate text from the model. This process is called decoding or sampling.

At each step, the LLM outputs a **probability distribution** over the next token. It is the job of the decoding algorithm to pick the next token, and move on to the next token. There are different decoding algorithms and hyper-parameters to control the generaiton:
* **Greedy** → pick the single highest‑probability token each step (safe but repetitive).  
* **Top‑k / Nucleus (top‑p)** → sample from a subset of likely tokens (adds variety).
* **beam** -> applies beam search to pick tokens
* **Temperature** → a *creativity* knob. Higher values flatten the probability distribution.

### 3.1 - Greedy decoding

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""
for k, v in MODELS.items():
  tokenizers[k] = AutoTokenizer.from_pretrained(v)
  models[k] = AutoModelForCausalLM.from_pretrained(v)


In [21]:

def generate(model_key, prompt, strategy="greedy", max_new_tokens=100):
    tok, mdl = tokenizers[model_key], models[model_key]
    # Return the generations based on the provided strategy: greedy, top_k, top_p
    """
    YOUR CODE HERE
    """
    with torch.no_grad():
      print("input: ", prompt)
      for i in range(max_new_tokens):
        tokens = tok.encode(prompt, return_tensors="pt")
        output = mdl(tokens)
        logits = output.logits
        last_token_logits = logits[0, -1, :]
        if strategy == "greedy":
          next_token = F.softmax(last_token_logits, dim=-1).argmax().item()
          next_token_decoded = tok.decode(next_token)
        elif strategy == "top-p":
          p = 0.9
          sorted_logits, sorted_indices = torch.sort(last_token_logits, descending=True)
          sorted_probs = F.softmax(sorted_logits, dim=-1)
          cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

          keep_mask = cumulative_probs <= p # [True,True,.....,False,False]
          keep_mask[0] = True # always keep the most likely token, so there's something to output even if probability of most likely one is above p
          filtered_probs = sorted_probs * keep_mask
          filtered_probs = filtered_probs / filtered_probs.sum() # renormalize

          sampled_idx = torch.multinomial(filtered_probs, num_samples=1).item()
          next_token_decoded = tok.decode(sorted_indices[sampled_idx])
        elif strategy == "top-k":
          k = 10
          sorted_logits, sorted_indices = torch.sort(last_token_logits, descending=True)
          sorted_probs = F.softmax(sorted_logits, dim=-1)
          top_k_tokens = torch.topk(sorted_probs, k=k)

          # renormalize top k and then pick based on probabilities
          top_k_probs = top_k_tokens.values / top_k_tokens.values.sum()
          sampled_idx = torch.multinomial(top_k_probs, num_samples=1).item()
          next_token_decoded = tok.decode(top_k_tokens.indices[sampled_idx])

        prompt += next_token_decoded

    return prompt


In [17]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Greedy ==")
    print(generate("gpt2", prompt, "greedy", 80))



== GPT-2 | Greedy ==
input:  Once upon a time
Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

== GPT-2 | Greedy ==
input:  What is 2+2?
What is 2+2?


2+2 is a term used to describe a number that is greater than or equal to 2.


2+2 is a number that is greater than or equal to 2.


2+2 is a number that is greater than or equal to 2.


2+2 is a number that is greater than or equal to 2.


2+2 is

== GPT-2 | Greedy ==
input:  Suggest a party theme.
Suggest a party theme.


The party theme is a simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, simple, si


Naively picking the single best token every time has the following issues in practice:

* **Loop**: “The cat is is is…”  
* **Miss long-term payoff**: the highest-probability word *now* might paint you into a boring corner later.

### 3.2 - Top-k or top-p sampling

In [18]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate("gpt2", prompt, "top-p", 40))



== GPT-2 | Top-p ==
input:  Once upon a time
Once upon a time they were taken to PortCobain, second only to the now deserted port of Dovahr. The latter was also besieged by them, and was destroyed, not for its inhabitants but for

== GPT-2 | Top-p ==
input:  What is 2+2?
What is 2+2?


The gray whale (Kyoto: Choppers) hibernates in the middle of the night and struggles to train. The kookaburra knows how to do things that are

== GPT-2 | Top-p ==
input:  Suggest a party theme.
Suggest a party theme.


Flat Bunch Showings: Available on Wednesdays, Saturdays and Sundays during 2016-17 season


Location: Moab - The Maze, South Middleton, Michigan


### 3.3 - Try It Yourself

1. Scroll to the list called `tests`.
2. Swap in your own prompts or tweak the decoding strategy.  
3. Re‑run the cell and compare the vibes.

> **Tip:** Try the same prompt with `greedy` vs. `top_p` (0.9) and see how the tone changes. Notice especially how small temperature tweaks can soften or sharpen the prose.

* `strategy`: `"greedy"`, `"beam"`, `"top_k"`, `"top_p"`  
* `temperature`: `0.2 – 2.0`  
* `k` or `p` thresholds



# 4 - Completion vs. Instruction-tuned LLMs

We have seen that we can use GPT2 model to pass an input text and generate a new text. However, this model only continues the provided text. It is not engaging in a dialouge-like conversation and cannot be helpful by answering instructions. On the other hand, **instruction-tuned LLMs** like `Qwen-Chat` go through an extra training stage called **post-training** after the base “completion” model is finished. Because of post-training step, an instruction-tuned LLM will:

* **Read the entire prompt as a request,** not just as text to mimic.  
* **Stay in dialogue mode**. Answer questions, follow steps, ask clarifying queries.  
* **Refuse or safe-complete** when instructions are unsafe or disallowed.  
* **Adopt a consistent persona** (e.g., “Assistant”) rather than drifting into story continuation.


### 4.1 - Qwen1.5-8B vs. GPT2

In the code below we’ll feed the same prompt to:

* **GPT-2 (completion-only)** – it will simply keep writing in the same style.  
* **Qwen-Chat (instruction-tuned)** – it will obey the instruction and respond directly.

Comparing the two outputs makes the difference easy to see.

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODELS = {
    "gpt2": "gpt2",
    "qwen": "Qwen/Qwen1.5-1.8B-Chat"
}
tokenizers, models = {}, {}
# Load models and tokenizers
"""
YOUR CODE HERE
"""
for k, v in MODELS.items():
  tokenizers[k] = AutoTokenizer.from_pretrained(v)
  models[k] = AutoModelForCausalLM.from_pretrained(v)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]


We downloaded two tiny checkpoints: `GPT‑2` (124 M parameters) and `Qwen‑1.5‑Chat` (1.8 B). If the cell took a while, that was mostly network time. Models are stored locally after the first run.

Let's now generate text and compare two models.


In [ ]:
tests=[("Once upon a time","greedy"),("What is 2+2?","top-k"),("Suggest a party theme.","top-p")]
for prompt,strategy in tests[1:]:
    for key in ["gpt2","qwen"]:
        print(f"\n== {key.upper()} | {strategy} ==")
        print(generate(key,prompt,strategy,80))



== GPT2 | top-k ==
input:  What is 2+2?


# 5. (Optional) A Small LLM Playground

### 5.1 ‑ Interactive Playground

Enter a prompt, pick a model and decoding strategy, adjust the temperature, and press **Generate** to watch the model respond.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Make sure models and tokenizers are loaded
try:
    tokenizers
    models
except NameError:
    raise RuntimeError("Please run the earlier setup cells that load the models before using the playground.")

def generate_playground(model_key, prompt, strategy="greedy", temperature=1.0, max_new_tokens=100):
    # Generation code
    """
    YOUR CODE HERE
    """

# Your code to build boxes, dropdowns, and other elements in the UI using widgets and creating the UI using widgets.vbox and display.
# Refer to https://ipywidgets.readthedocs.io/en/stable/
"""
YOUR CODE HERE
"""



## 🎉 Congratulations!

You’ve just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen1.5-8B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
